In [ ]:
!huggingface-cli login

# ContactDoctor Bio Medical LLM

In [ ]:
!pip install -q transformers accelerate bitsandbytes gradio

In [ ]:
!pip install peft

In [ ]:
!pip install -q datasets

### Load the data from the file to train the model

In [ ]:
from datasets import Dataset
data = [  ]
dataset = Dataset.from_list(data)

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from datasets import Dataset
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType

torch.cuda.empty_cache()
# === Step 1: Model & Tokenizer Setup ===
model_id = "AnjaliNV/WellBeing_LLM"  # or base model like "mistralai/Mistral-7B-Instruct-v0.1"

print("🔄 Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token  # Required for some models

print("📂 Loading local dataset...")
# dataset = load_dataset("json", data_files="/content/fitness-chat-prompt-completion-dataset.json")  # Update path
# Using the dataset created in the previous cell


# Tokenize function
# def preprocess(batch):
#     texts = [f"### Question: {p}\n### Answer: {c}" for p, c in zip(batch["instruction"], batch["output"])]
#     tokens = tokenizer(texts, truncation=True, padding="max_length", max_length=512)
#     tokens["labels"] = tokens["input_ids"].copy()
#     return tokens


def tokenize_function(examples):
    # Concatenate prompt + completion as text to train on
    texts = [p + c for p, c in zip(examples["prompt"], examples["completion"])]
    return tokenizer(texts, padding="max_length", truncation=True, max_length=512)





print("🔄 Tokenizing dataset...")
# Process without batching to avoid potential issues with padding/truncation in batches
# tokenized_dataset = dataset["train"].map(preprocess, batched=True)
tokenized_dataset = dataset.map(tokenize_function, batched=True)



# === Step 2: Load Quantized Base Model ===
print("🔄 Loading base model...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)


model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    quantization_config=bnb_config,
    trust_remote_code=True
)

# === Step 3: Inject LoRA ===
print("🧠 Adding LoRA adapters...")
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# === Step 4: Set Up Trainer ===
print("⚙️ Preparing trainer...")
training_args = TrainingArguments(
    output_dir="./WellBeing_LLM_finetuned",
    per_device_train_batch_size=1,      # lowered batch size
    gradient_accumulation_steps=8,      # to keep effective batch size
    num_train_epochs=3,
    logging_steps=10,
    save_steps=100,
    save_total_limit=1,
    learning_rate=3e-4,
    fp16=True,
    optim="paged_adamw_8bit",
    report_to=[],
)


data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# === Step 5: Train ===
print("🚀 Starting training...")
trainer.train()

# === Step 6: Save Final Model + Tokenizer ===
print("💾 Saving the fine-tuned model...")
trainer.save_model("./WellBeing_LLM_finetuned")
tokenizer.save_pretrained("./WellBeing_LLM_finetuned")

In [ ]:
from peft import PeftModel
from transformers import AutoTokenizer
from huggingface_hub import notebook_login

# Optional: If running in Colab/Jupyter
# notebook_login()

repo_name = "AnjaliNV/WellBeing_LLM"  # Choose a name

# Push the LoRA adapter
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import torch

base_model_id = "ContactDoctor/Bio-Medical-Llama-3-8B"
adapter_path = "AnjaliNV/WellBeing_LLM"

# Optional: Enable 4-bit quantization to reduce memory usage
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)

# Load model with quantization and low memory footprint
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    quantization_config=bnb_config,
    trust_remote_code=True
)

# Load the LoRA adapter (AFTER the base model is on device)
model = PeftModel.from_pretrained(model, adapter_path)
model.eval()


## RAG fine tuning

In [ ]:
!pip install langchain faiss-cpu sentence-transformers transformers


In [ ]:
!pip install PyPDF2

In [ ]:
!pip install -U langchain-community

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = FAISS.load_local(
    "/content/drive/MyDrive/rag_index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)


In [ ]:
def load_and_split_document(filepath):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    import PyPDF2

    text = ""

    if filepath.endswith(".txt"):
        with open(filepath, "r", encoding="utf-8") as f:
            text = f.read()
    elif filepath.endswith(".pdf"):
        with open(filepath, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            for page in reader.pages:
                text += page.extract_text() + "\n"
    else:
        raise ValueError("Unsupported file format")

    # Split text into chunks
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    return splitter.split_text(text)

new_chunks = load_and_split_document("/content/physical_exercise2.pdf")  # or .txt
vectorstore.add_texts(new_chunks)
vectorstore.save_local("/content/drive/MyDrive/rag_index")
print("Vectorstore updated and saved.")


In [ ]:
import gradio as gr
import torch
import re

def chat_rag_interface(message, history):
    # Step 1: Retrieve relevant context chunks for the user question
    retrieved_chunks = vectorstore.similarity_search(message, k=3)

    # Step 2: Combine chunks into one context string with better formatting
    context = "\n---\n".join([doc.page_content for doc in retrieved_chunks])

    # Step 3: Improved prompt template with clearer instructions
    prompt_template = f"""<|system|>
You are a wellbeing expert. You MUST follow the exact format shown in the example below. Do NOT deviate from this structure.

IMPORTANT RULES:
1. Always respond with exactly 4 sections: Food Recommendation, Physical Exercise, Sleep and Recovery, Overall Suggestion
2. Use bullet points with a), b), c), d) format as shown
3. Base recommendations on the Zone Minutes logic provided
4. Keep responses factual and avoid making up specific numbers or data not provided
5. End your response with "### End ###"

Zone Minutes Guidelines:
- Zone Minutes < 90: Recommend starting with low-impact activities
- Zone Minutes 90–150: Encourage maintaining current activity with gradual progression
- Zone Minutes > 150: Recommend advanced workouts and advise on recovery

CONTEXT:
{context}

EXAMPLE FORMAT (FOLLOW EXACTLY):
User Data: Total Steps: 100,000, Average Zone Minutes: 160, Average Heart Rate: 60 bpm, Food Consumption: healthy, Sleep: 7.5 hrs average, Physical Activities: running, gym

Response:
1. Food Recommendation:
   Your food intake is already healthy, try different varieties of food
   a) Protein Intake: Include lean meats, fish, tofu, or legumes to support muscle repair
   b) Healthy Fats: Add avocados, olive oil, almonds, walnuts, and chia seeds
   c) Vegetable Variety: Eat spinach, broccoli, carrots, and zucchini for essential vitamins
   d) Hydration: Aim for 3–4 liters of water per day to stay well-hydrated

2. Physical Exercise:
   You have good cardiovascular fitness from running. Along with walking you can include:
   a) Strength training: squats, deadlifts, bench press, and overhead press
   b) Yoga or Pilates for flexibility and core strength
   c) Outdoor Activities/Adventure Sports: Cycling, tracking, swimming

3. Sleep and Recovery:
   Aim for 7–8 hours of sleep. Avoid screens before bed and follow a consistent schedule.

4. Overall Suggestion:
   Great job! Keep up the consistency. You could also explore group fitness or meditation for balance.

### End ###
<|/system|>

<|user|>
User Data: {message}
<|/user|>

<|assistant|>
Response:
"""

    # Step 4: Improved tokenization and generation parameters
    inputs = tokenizer(prompt_template, return_tensors="pt", truncation=True, max_length=2048).to(model.device)

    # Better generation parameters to reduce hallucination
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=400,  # Reduced to prevent over-generation
            min_new_tokens=50,   # Ensure minimum response length
            temperature=0.3,     # Lower temperature for more focused responses
            top_p=0.85,          # Slightly lower for better coherence
            top_k=40,            # Add top_k sampling
            do_sample=True,
            repetition_penalty=1.1,  # Reduced repetition penalty
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=[
                tokenizer.encode("### End ###")[0] if "### End ###" in tokenizer.get_vocab() else tokenizer.eos_token_id,
                tokenizer.eos_token_id
            ],
            early_stopping=True,
            no_repeat_ngram_size=3  # Prevent repetitive phrases
        )

    # Step 5: Better response extraction and cleaning
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the assistant's response
    if "<|assistant|>" in response:
        generated_text = response.split("<|assistant|>")[-1].strip()
    else:
        # Fallback: remove the prompt template
        generated_text = response[len(prompt_template):].strip()

    # Clean up the response
    generated_text = clean_response(generated_text)

    # Validate response format
    if not validate_response_format(generated_text):
        # If format is invalid, try to fix it or return a structured fallback
        generated_text = fix_response_format(generated_text, message)

    # Optional: Save chat history
    with open("chat_history.txt", "a", encoding="utf-8") as f:
        f.write(f"User: {message}\n")
        f.write(f"Model: {generated_text}\n\n")

    return generated_text

def clean_response(text):
    """Clean and format the model response"""
    # Remove any remaining prompt artifacts
    text = re.sub(r'<\|.*?\|>', '', text)
    text = re.sub(r'User Data:.*?Response:', '', text, flags=re.DOTALL)

    # Ensure proper formatting
    text = text.strip()

    # Remove multiple consecutive newlines
    text = re.sub(r'\n{3,}', '\n\n', text)

    return text

def validate_response_format(text):
    """Check if response follows the required format"""
    required_sections = [
        "1. Food Recommendation",
        "2. Physical Exercise",
        "3. Sleep and Recovery",
        "4. Overall Suggestion"
    ]

    for section in required_sections:
        if section not in text:
            return False

    return True

def fix_response_format(text, user_message):
    """Attempt to fix malformed responses or provide fallback"""
    # Extract zone minutes from user message if possible
    zone_minutes_match = re.search(r'zone minutes?:?\s*(\d+)', user_message.lower())
    zone_minutes = int(zone_minutes_match.group(1)) if zone_minutes_match else 100

    # Create a properly formatted fallback response
    if zone_minutes < 90:
        exercise_rec = "Start with low-impact activities like walking, light stretching, and basic bodyweight exercises"
    elif zone_minutes <= 150:
        exercise_rec = "Maintain your current activity level with gradual progression in intensity and duration"
    else:
        exercise_rec = "Focus on advanced workouts while prioritizing recovery and rest days"

    fallback_response = f"""1. Food Recommendation:
   Focus on a balanced diet to support your fitness goals
   a) Protein Intake: Include lean proteins to support muscle recovery
   b) Complex Carbohydrates: Add whole grains and vegetables for sustained energy
   c) Healthy Fats: Include nuts, seeds, and olive oil for overall health
   d) Hydration: Maintain adequate water intake throughout the day

2. Physical Exercise:
   {exercise_rec}
   a) Cardiovascular Training: Include activities that elevate your heart rate
   b) Strength Training: Add resistance exercises for muscle development
   c) Flexibility Work: Include stretching or yoga for mobility

3. Sleep and Recovery:
   Prioritize 7-9 hours of quality sleep each night for optimal recovery

4. Overall Suggestion:
   Maintain consistency in your wellness routine and listen to your body's needs

### End ###"""

    return fallback_response

In [ ]:
gr.ChatInterface(
    fn=chat_rag_interface,
    title="Well Being Advicer",
    description="Receive holistic health advice grounded in domain knowledge.",
).queue().launch(debug=True, share=True)


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the saved model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("WellBeing_LLM")
tokenizer = AutoTokenizer.from_pretrained("WellBeing_LLM")

# Push to HF Hub
model.push_to_hub("WellBeing_LLM")        # this creates a repo under your username
tokenizer.push_to_hub("WellBeing_LLM")


# Data structure and Memory System

In [ ]:
# ========================================================================
# STEP 1: Data Structures and Memory System
# ========================================================================
import json
import os
from datetime import datetime, timedelta
from typing import Dict, List, Optional
import pandas as pd
import numpy as np

class UserProfile:
    """Store user's basic info, targets, and preferences"""

    def __init__(self, user_id: str):
        self.user_id = user_id
        self.created_date = datetime.now().strftime('%Y-%m-%d')

        # Default targets (can be customized per user)
        self.targets = {
            'daily_steps': 10000,
            'weekly_zone_minutes': 150,
            'daily_sleep_hours': 8,
            'weekly_exercise_sessions': 3,
            'daily_calories': 2000
        }

        # User preferences and constraints
        self.preferences = {
            'preferred_exercises': [],
            'food_restrictions': [],
            'schedule_constraints': {},
            'health_conditions': []
        }

        # Learning data
        self.response_patterns = {
            'follows_exercise_recs': 0.5,  # How often they follow exercise advice
            'follows_nutrition_recs': 0.5,
            'follows_sleep_recs': 0.5,
            'preferred_rec_types': []
        }

    def update_targets(self, new_targets: Dict):
        """Update user targets"""
        self.targets.update(new_targets)

    def update_preferences(self, new_prefs: Dict):
        """Update user preferences"""
        self.preferences.update(new_prefs)

    def to_dict(self):
        return {
            'user_id': self.user_id,
            'created_date': self.created_date,
            'targets': self.targets,
            'preferences': self.preferences,
            'response_patterns': self.response_patterns
        }

class WeeklyHealthData:
    """Store one week's health data"""

    def __init__(self, user_id: str, week_start: str, data: Dict):
        self.user_id = user_id
        self.week_start = week_start
        self.week_number = self._calculate_week_number(week_start)

        # Raw weekly data
        self.total_steps = data.get('total_steps', 0)
        self.zone_minutes = data.get('zone_minutes', 0)
        self.sleep_hours = data.get('sleep_hours', [])  # Daily values
        self.exercise_sessions = data.get('exercise_sessions', [])
        self.mood_scores = data.get('mood_scores', [])
        self.food_data = data.get('food_data', [])

        # Calculated metrics
        self.avg_daily_steps = self.total_steps / 7
        self.avg_sleep = np.mean(self.sleep_hours) if self.sleep_hours else 0
        self.avg_mood = np.mean(self.mood_scores) if self.mood_scores else 0
        self.total_calories = sum([food.get('calories', 0) for food in self.food_data])
        self.avg_daily_calories = self.total_calories / 7

        # Timestamp
        self.created_at = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    def _calculate_week_number(self, week_start: str) -> int:
        """Calculate week number since user started"""
        week_date = datetime.strptime(week_start, '%Y-%m-%d')
        start_of_year = datetime(week_date.year, 1, 1)
        return ((week_date - start_of_year).days // 7) + 1

    def to_dict(self):
        return {
            'user_id': self.user_id,
            'week_start': self.week_start,
            'week_number': self.week_number,
            'total_steps': self.total_steps,
            'zone_minutes': self.zone_minutes,
            'sleep_hours': self.sleep_hours,
            'exercise_sessions': self.exercise_sessions,
            'mood_scores': self.mood_scores,
            'food_data': self.food_data,
            'avg_daily_steps': self.avg_daily_steps,
            'avg_sleep': self.avg_sleep,
            'avg_mood': self.avg_mood,
            'total_calories': self.total_calories,
            'avg_daily_calories': self.avg_daily_calories,
            'created_at': self.created_at
        }

class WeeklyRecommendations:
    """Store recommendations given for a specific week"""

    def __init__(self, user_id: str, week_start: str, recommendations: Dict):
        self.user_id = user_id
        self.week_start = week_start
        self.recommendations = recommendations
        self.created_at = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        # Track user response to recommendations
        self.user_feedback = {
            'followed_exercise': None,
            'followed_nutrition': None,
            'followed_sleep': None,
            'difficulty_level': None,  # 1-5 scale
            'effectiveness': None,     # 1-5 scale
            'notes': ""
        }

    def update_feedback(self, feedback: Dict):
        """Update user feedback on recommendations"""
        self.user_feedback.update(feedback)

    def to_dict(self):
        return {
            'user_id': self.user_id,
            'week_start': self.week_start,
            'recommendations': self.recommendations,
            'user_feedback': self.user_feedback,
            'created_at': self.created_at
        }

class HealthDataManager:
    """Manage all user health data and provide persistence"""

    def __init__(self, data_dir: str = "health_data"):
        self.data_dir = data_dir
        self._ensure_directories()

    def _ensure_directories(self):
        """Create necessary directories"""
        os.makedirs(f"{self.data_dir}/profiles", exist_ok=True)
        os.makedirs(f"{self.data_dir}/weekly_data", exist_ok=True)
        os.makedirs(f"{self.data_dir}/recommendations", exist_ok=True)

    def save_user_profile(self, profile: UserProfile):
        """Save user profile to file"""
        file_path = f"{self.data_dir}/profiles/{profile.user_id}.json"
        with open(file_path, 'w') as f:
            json.dump(profile.to_dict(), f, indent=2)

    def load_user_profile(self, user_id: str) -> Optional[UserProfile]:
        """Load user profile from file"""
        file_path = f"{self.data_dir}/profiles/{user_id}.json"
        if os.path.exists(file_path):
            with open(file_path, 'r') as f:
                data = json.load(f)
                profile = UserProfile(user_id)
                profile.targets = data['targets']
                profile.preferences = data['preferences']
                profile.response_patterns = data['response_patterns']
                profile.created_date = data['created_date']
                return profile
        return None

    def save_weekly_data(self, weekly_data: WeeklyHealthData):
        """Save weekly health data"""
        user_dir = f"{self.data_dir}/weekly_data/{weekly_data.user_id}"
        os.makedirs(user_dir, exist_ok=True)

        file_path = f"{user_dir}/{weekly_data.week_start}.json"
        with open(file_path, 'w') as f:
            json.dump(weekly_data.to_dict(), f, indent=2)

    def load_user_weekly_data(self, user_id: str, num_weeks: int = 4) -> List[WeeklyHealthData]:
        """Load recent weekly data for a user"""
        user_dir = f"{self.data_dir}/weekly_data/{user_id}"
        if not os.path.exists(user_dir):
            return []

        # Get all weekly data files
        files = [f for f in os.listdir(user_dir) if f.endswith('.json')]
        files.sort(reverse=True)  # Most recent first

        weekly_data_list = []
        for file in files[:num_weeks]:
            file_path = f"{user_dir}/{file}"
            with open(file_path, 'r') as f:
                data = json.load(f)
                weekly_data = WeeklyHealthData(
                    data['user_id'],
                    data['week_start'],
                    data
                )
                weekly_data_list.append(weekly_data)

        return weekly_data_list

    def save_recommendations(self, recommendations: WeeklyRecommendations):
        """Save weekly recommendations"""
        user_dir = f"{self.data_dir}/recommendations/{recommendations.user_id}"
        os.makedirs(user_dir, exist_ok=True)

        file_path = f"{user_dir}/{recommendations.week_start}.json"
        with open(file_path, 'w') as f:
            json.dump(recommendations.to_dict(), f, indent=2)

    def load_user_recommendations(self, user_id: str, num_weeks: int = 4) -> List[WeeklyRecommendations]:
        """Load recent recommendations for a user"""
        user_dir = f"{self.data_dir}/recommendations/{user_id}"
        if not os.path.exists(user_dir):
            return []

        files = [f for f in os.listdir(user_dir) if f.endswith('.json')]
        files.sort(reverse=True)

        rec_list = []
        for file in files[:num_weeks]:
            file_path = f"{user_dir}/{file}"
            with open(file_path, 'r') as f:
                data = json.load(f)
                rec = WeeklyRecommendations(
                    data['user_id'],
                    data['week_start'],
                    data['recommendations']
                )
                rec.user_feedback = data['user_feedback']
                rec.created_at = data['created_at']
                rec_list.append(rec)

        return rec_list

    def get_user_progress_summary(self, user_id: str) -> Dict:
        """Get overall progress summary for a user"""
        weekly_data = self.load_user_weekly_data(user_id, num_weeks=8)

        if not weekly_data:
            return {"message": "No data available"}

        # Calculate trends
        weeks = len(weekly_data)

        # Steps trend
        steps_trend = [w.avg_daily_steps for w in reversed(weekly_data)]

        # Sleep trend
        sleep_trend = [w.avg_sleep for w in reversed(weekly_data)]

        # Zone minutes trend
        zone_trend = [w.zone_minutes for w in reversed(weekly_data)]

        # Mood trend
        mood_trend = [w.avg_mood for w in reversed(weekly_data)]

        return {
            'weeks_tracked': weeks,
            'steps_trend': steps_trend,
            'sleep_trend': sleep_trend,
            'zone_trend': zone_trend,
            'mood_trend': mood_trend,
            'latest_week': weekly_data[0].to_dict() if weekly_data else None
        }

# ========================================================================
# STEP 1 USAGE EXAMPLE
# ========================================================================

def example_usage():
    """Example of how to use the data management system"""

    # Initialize data manager
    data_manager = HealthDataManager()

    # Create or load user profile
    user_id = "user_123"
    profile = data_manager.load_user_profile(user_id)

    if not profile:
        # Create new user
        profile = UserProfile(user_id)
        profile.update_targets({
            'daily_steps': 12000,  # Custom target
            'weekly_zone_minutes': 180
        })
        profile.update_preferences({
            'preferred_exercises': ['running', 'yoga'],
            'food_restrictions': ['gluten-free']
        })
        data_manager.save_user_profile(profile)
        print(f"Created new user profile for {user_id}")
    else:
        print(f"Loaded existing profile for {user_id}")

    # Example weekly data
    week_data = {
        'total_steps': 68000,
        'zone_minutes': 140,
        'sleep_hours': [7.5, 6.8, 8.2, 7.0, 6.5, 8.5, 7.8],
        'exercise_sessions': [
            {'type': 'running', 'duration': 30, 'date': '2024-01-15'},
            {'type': 'yoga', 'duration': 45, 'date': '2024-01-17'}
        ],
        'mood_scores': [7, 6, 8, 7, 5, 8, 7],
        'food_data': [
            {'meal': 'breakfast', 'calories': 400, 'date': '2024-01-15'},
            {'meal': 'lunch', 'calories': 600, 'date': '2024-01-15'},
            # ... more food entries
        ]
    }

    # Save weekly data
    weekly_health_data = WeeklyHealthData(user_id, "2024-01-15", week_data)
    data_manager.save_weekly_data(weekly_health_data)
    print("Saved weekly health data")

    # Example recommendations
    recommendations = {
        'exercise': "Increase zone minutes by adding 2x20min cardio sessions",
        'nutrition': "Add protein-rich snacks between meals",
        'sleep': "Maintain current sleep schedule, it's working well",
        'overall': "Focus on cardiovascular fitness this week"
    }

    weekly_recs = WeeklyRecommendations(user_id, "2024-01-15", recommendations)
    data_manager.save_recommendations(weekly_recs)
    print("Saved weekly recommendations")

    # Get progress summary
    progress = data_manager.get_user_progress_summary(user_id)
    print(f"Progress summary: {progress}")

    return data_manager

if __name__ == "__main__":
    # Test the system
    data_manager = example_usage()

#Analytics Engine

In [ ]:
# ========================================================================
# STEP 2: Analytics Engine (Colab Version)
# ========================================================================
import numpy as np
from typing import Dict, List, Tuple
from datetime import datetime
import statistics

class HealthAnalytics:
    """Core analytics engine for health data analysis"""

    def __init__(self, data_manager):
        self.data_manager = data_manager

    def analyze_weekly_performance(self, user_id: str, current_week_data) -> Dict:
        """Comprehensive analysis of current week vs targets and trends"""

        # Load user profile and historical data
        profile = self.data_manager.load_user_profile(user_id)
        historical_data = self.data_manager.load_user_weekly_data(user_id, num_weeks=4)

        if not profile:
            return {"error": "User profile not found"}

        # Current week performance vs targets
        performance = self._calculate_target_performance(current_week_data, profile.targets)

        # Trend analysis
        trends = self._analyze_trends(current_week_data, historical_data)

        # Progress analysis
        progress = self._analyze_progress(current_week_data, historical_data)

        # Priority areas (what needs most attention)
        priorities = self._identify_priority_areas(performance, trends)

        return {
            'user_id': user_id,
            'week_start': current_week_data.week_start,
            'performance': performance,
            'trends': trends,
            'progress': progress,
            'priorities': priorities,
            'analysis_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }

    def _calculate_target_performance(self, current_data, targets: Dict) -> Dict:
        """Calculate performance against targets"""

        performance = {}

        # Steps performance
        steps_achievement = (current_data.avg_daily_steps / targets['daily_steps']) * 100
        performance['steps'] = {
            'current': current_data.avg_daily_steps,
            'target': targets['daily_steps'],
            'achievement_percent': min(steps_achievement, 150),  # Cap at 150%
            'status': self._get_performance_status(steps_achievement)
        }

        # Zone minutes performance
        zone_achievement = (current_data.zone_minutes / targets['weekly_zone_minutes']) * 100
        performance['zone_minutes'] = {
            'current': current_data.zone_minutes,
            'target': targets['weekly_zone_minutes'],
            'achievement_percent': min(zone_achievement, 150),
            'status': self._get_performance_status(zone_achievement)
        }

        # Sleep performance
        sleep_achievement = (current_data.avg_sleep / targets['daily_sleep_hours']) * 100
        performance['sleep'] = {
            'current': current_data.avg_sleep,
            'target': targets['daily_sleep_hours'],
            'achievement_percent': min(sleep_achievement, 120),  # Sleep shouldn't be too much over
            'status': self._get_performance_status(sleep_achievement)
        }

        # Exercise sessions
        exercise_achievement = (len(current_data.exercise_sessions) / targets['weekly_exercise_sessions']) * 100
        performance['exercise_sessions'] = {
            'current': len(current_data.exercise_sessions),
            'target': targets['weekly_exercise_sessions'],
            'achievement_percent': min(exercise_achievement, 150),
            'status': self._get_performance_status(exercise_achievement)
        }

        # Calories (if target is set)
        if targets.get('daily_calories'):
            calorie_achievement = (current_data.avg_daily_calories / targets['daily_calories']) * 100
            performance['calories'] = {
                'current': current_data.avg_daily_calories,
                'target': targets['daily_calories'],
                'achievement_percent': calorie_achievement,
                'status': self._get_calorie_status(calorie_achievement)
            }

        # Overall performance score
        scores = [perf['achievement_percent'] for perf in performance.values()]
        performance['overall'] = {
            'score': np.mean(scores),
            'status': self._get_performance_status(np.mean(scores))
        }

        return performance

    def _analyze_trends(self, current_data, historical_data: List) -> Dict:
        """Analyze trends over time"""

        if len(historical_data) < 2:
            return {"message": "Insufficient data for trend analysis"}

        # Include current week in analysis
        all_data = [current_data] + historical_data
        all_data.sort(key=lambda x: x.week_start)  # Sort chronologically

        trends = {}

        # Steps trend
        steps_data = [w.avg_daily_steps for w in all_data]
        trends['steps'] = self._calculate_trend(steps_data, "steps/day")

        # Zone minutes trend
        zone_data = [w.zone_minutes for w in all_data]
        trends['zone_minutes'] = self._calculate_trend(zone_data, "minutes/week")

        # Sleep trend
        sleep_data = [w.avg_sleep for w in all_data]
        trends['sleep'] = self._calculate_trend(sleep_data, "hours/night")

        # Mood trend
        mood_data = [w.avg_mood for w in all_data if w.avg_mood > 0]
        if mood_data:
            trends['mood'] = self._calculate_trend(mood_data, "mood score")

        return trends

    def _analyze_progress(self, current_data, historical_data: List) -> Dict:
        """Analyze progress compared to previous weeks"""

        if not historical_data:
            return {"message": "No historical data for comparison"}

        # Compare with last week
        last_week = historical_data[0]  # Most recent week

        progress = {}

        # Steps progress
        steps_change = current_data.avg_daily_steps - last_week.avg_daily_steps
        progress['steps'] = {
            'change': steps_change,
            'percent_change': (steps_change / last_week.avg_daily_steps) * 100 if last_week.avg_daily_steps > 0 else 0,
            'direction': 'improved' if steps_change > 0 else 'declined' if steps_change < 0 else 'maintained'
        }

        # Zone minutes progress
        zone_change = current_data.zone_minutes - last_week.zone_minutes
        progress['zone_minutes'] = {
            'change': zone_change,
            'percent_change': (zone_change / last_week.zone_minutes) * 100 if last_week.zone_minutes > 0 else 0,
            'direction': 'improved' if zone_change > 0 else 'declined' if zone_change < 0 else 'maintained'
        }

        # Sleep progress
        sleep_change = current_data.avg_sleep - last_week.avg_sleep
        progress['sleep'] = {
            'change': sleep_change,
            'percent_change': (sleep_change / last_week.avg_sleep) * 100 if last_week.avg_sleep > 0 else 0,
            'direction': 'improved' if sleep_change > 0 else 'declined' if sleep_change < 0 else 'maintained'
        }

        # Mood progress
        if current_data.avg_mood > 0 and last_week.avg_mood > 0:
            mood_change = current_data.avg_mood - last_week.avg_mood
            progress['mood'] = {
                'change': mood_change,
                'percent_change': (mood_change / last_week.avg_mood) * 100,
                'direction': 'improved' if mood_change > 0 else 'declined' if mood_change < 0 else 'maintained'
            }

        return progress

    def _identify_priority_areas(self, performance: Dict, trends: Dict) -> List[Dict]:
        """Identify which areas need most attention"""

        priorities = []

        # Check each area
        for area, perf in performance.items():
            if area == 'overall':
                continue

            priority_score = 0
            reasons = []

            # Low performance
            if perf['achievement_percent'] < 70:
                priority_score += 3
                reasons.append(f"Below target ({perf['achievement_percent']:.1f}%)")
            elif perf['achievement_percent'] < 85:
                priority_score += 2
                reasons.append(f"Slightly below target ({perf['achievement_percent']:.1f}%)")

            # Declining trend
            if area in trends and trends[area].get('direction') == 'declining':
                priority_score += 2
                reasons.append("Declining trend")

            # Add to priorities if score is significant
            if priority_score > 0:
                priorities.append({
                    'area': area,
                    'priority_score': priority_score,
                    'reasons': reasons,
                    'current_performance': perf['achievement_percent']
                })

        # Sort by priority score (highest first)
        priorities.sort(key=lambda x: x['priority_score'], reverse=True)

        return priorities

    def _calculate_trend(self, data: List[float], unit: str) -> Dict:
        """Calculate trend direction and magnitude"""

        if len(data) < 2:
            return {"direction": "insufficient_data"}

        # Calculate linear trend
        x = np.arange(len(data))
        slope, intercept = np.polyfit(x, data, 1)

        # Determine trend direction
        if slope > 0.1:
            direction = "improving"
        elif slope < -0.1:
            direction = "declining"
        else:
            direction = "stable"

        # Calculate percentage change from first to last
        if data[0] != 0:
            percent_change = ((data[-1] - data[0]) / data[0]) * 100
        else:
            percent_change = 0

        return {
            'direction': direction,
            'slope': slope,
            'percent_change': percent_change,
            'recent_value': data[-1],
            'unit': unit,
            'data_points': len(data)
        }

    def _get_performance_status(self, achievement_percent: float) -> str:
        """Get performance status based on achievement percentage"""
        if achievement_percent >= 100:
            return "excellent"
        elif achievement_percent >= 85:
            return "good"
        elif achievement_percent >= 70:
            return "fair"
        else:
            return "needs_improvement"

    def _get_calorie_status(self, achievement_percent: float) -> str:
        """Get calorie status (different logic as too many calories is bad)"""
        if 90 <= achievement_percent <= 110:
            return "excellent"
        elif 80 <= achievement_percent <= 120:
            return "good"
        elif 70 <= achievement_percent <= 130:
            return "fair"
        else:
            return "needs_adjustment"

    def generate_insights(self, analysis: Dict) -> List[str]:
        """Generate human-readable insights from analysis"""

        insights = []

        # Overall performance insight
        overall_score = analysis['performance']['overall']['score']
        if overall_score >= 90:
            insights.append("🎉 Excellent overall performance this week!")
        elif overall_score >= 75:
            insights.append("👍 Good overall performance with room for improvement")
        else:
            insights.append("⚠️ Several areas need attention this week")

        # Priority area insights
        priorities = analysis['priorities']
        if priorities:
            top_priority = priorities[0]
            insights.append(f"🎯 Top priority: {top_priority['area'].replace('_', ' ').title()}")

        # Trend insights - FIXED
        trends = analysis.get('trends', {})
        improving_areas = []
        declining_areas = []

        # Check if trends is a dictionary before iterating
        if isinstance(trends, dict):
            for area, trend in trends.items():
                # Check if trend is a dictionary (not a string)
                if isinstance(trend, dict):
                    if trend.get('direction') == 'improving':
                        improving_areas.append(area)
                    elif trend.get('direction') == 'declining':
                        declining_areas.append(area)
                elif isinstance(trend, str):
                    # Handle case where trend is just a string
                    if trend == 'improving':
                        improving_areas.append(area)
                    elif trend == 'declining':
                        declining_areas.append(area)

        if improving_areas:
            insights.append(f"📈 Improving: {', '.join(improving_areas)}")

        if declining_areas:
            insights.append(f"📉 Needs attention: {', '.join(declining_areas)}")

        # Progress insights - FIXED
        progress = analysis.get('progress', {})
        if isinstance(progress, dict):
            for area, prog in progress.items():
                if isinstance(prog, dict):
                    if prog.get('direction') == 'improved' and prog.get('percent_change', 0) > 10:
                        insights.append(f"🚀 Great improvement in {area}: +{prog['percent_change']:.1f}%")

        return insights

# ========================================================================
# CORRECTED USAGE EXAMPLE FOR COLAB
# ========================================================================

def example_analytics():
    """Example of using the analytics engine - CORRECTED FOR COLAB"""

    # Use the data manager and classes from the previous cell (Step 1)
    # No import needed since they're already defined in the notebook

    data_manager = HealthDataManager()
    analytics = HealthAnalytics(data_manager)

    # Example current week data
    current_week_data = WeeklyHealthData("user_123", "2024-01-22", {
        'total_steps': 72000,  # Improved from last week
        'zone_minutes': 160,   # Good improvement
        'sleep_hours': [7.8, 7.2, 8.1, 7.5, 7.0, 8.3, 7.9],
        'exercise_sessions': [
            {'type': 'running', 'duration': 35},
            {'type': 'yoga', 'duration': 45},
            {'type': 'cycling', 'duration': 40}
        ],
        'mood_scores': [8, 7, 8, 7, 6, 9, 8],
        'food_data': []  # Simplified for example
    })

    # Perform analysis
    analysis = analytics.analyze_weekly_performance("user_123", current_week_data)

    # Generate insights
    insights = analytics.generate_insights(analysis)

    print("=== WEEKLY HEALTH ANALYSIS ===")
    print(f"User: {analysis['user_id']}")
    print(f"Week: {analysis['week_start']}")
    print(f"Overall Score: {analysis['performance']['overall']['score']:.1f}/100")

    print("\n=== KEY INSIGHTS ===")
    for insight in insights:
        print(insight)

    print("\n=== PERFORMANCE BREAKDOWN ===")
    for area, perf in analysis['performance'].items():
        if area != 'overall':
            print(f"{area.replace('_', ' ').title()}: {perf['achievement_percent']:.1f}% ({perf['status']})")

    if analysis['priorities']:
        print("\n=== PRIORITY AREAS ===")
        for priority in analysis['priorities'][:3]:  # Top 3
            print(f"{priority['area'].replace('_', ' ').title()}: {', '.join(priority['reasons'])}")

    return analysis

# Test the analytics (optional - run this to test)
if __name__ == "__main__":
    analysis = example_analytics()

In [ ]:
# ========================================================================
# STEP 3: LLM Integration with Your Health Model
# ========================================================================
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import torch
from langchain.vectorstores import FAISS
# Updated import to fix deprecation warning
try:
    from langchain_huggingface import HuggingFaceEmbeddings
except ImportError:
    from langchain.embeddings import HuggingFaceEmbeddings
import re
from typing import Dict, List
import json

class HealthLLMProcessor:
    """Enhanced LLM processor for health analytics with memory and learning"""

    def __init__(self, base_model_id: str, adapter_path: str, vector_db_path: str, data_manager):
        self.data_manager = data_manager
        self._load_model(base_model_id, adapter_path)
        self._load_vector_db(vector_db_path)

    def _load_model(self, base_model_id: str, adapter_path: str):
        """Load your fine-tuned model"""
        print("Loading health LLM...")

        # Your exact configuration
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.float16,
        )

        # Load tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        # Load model with quantization
        self.model = AutoModelForCausalLM.from_pretrained(
            base_model_id,
            device_map="auto",
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True,
            quantization_config=bnb_config,
            trust_remote_code=True
        )

        # Load the LoRA adapter
        self.model = PeftModel.from_pretrained(self.model, adapter_path)
        self.model.eval()
        print("✅ Model loaded successfully")

    def _load_vector_db(self, vector_db_path: str):
        """Load your vector database"""
        print("Loading vector database...")
        self.embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
        self.vectorstore = FAISS.load_local(
            vector_db_path,
            embeddings=self.embedding_model,
            allow_dangerous_deserialization=True
        )
        print("✅ Vector database loaded successfully")

    def generate_weekly_recommendations(self, user_id: str, analysis_data: Dict) -> Dict:
        """Generate comprehensive weekly recommendations based on analysis"""

        # Get user context
        user_context = self._build_user_context(user_id)

        # Get relevant knowledge from vector DB
        knowledge_context = self._get_relevant_knowledge(analysis_data)

        # Build prompt with all context
        prompt = self._build_comprehensive_prompt(user_id, analysis_data, user_context, knowledge_context)

        # Generate recommendations
        raw_response = self._generate_response(prompt)

        # Parse and structure the response
        structured_recommendations = self._parse_recommendations(raw_response)

        # Learn from this interaction
        self._update_user_learning(user_id, analysis_data, structured_recommendations)

        return structured_recommendations

    def _build_user_context(self, user_id: str) -> str:
        """Build context about user's history and preferences"""

        # Get user profile
        profile = self.data_manager.load_user_profile(user_id)

        # Get recent recommendations and feedback
        recent_recs = self.data_manager.load_user_recommendations(user_id, num_weeks=2)

        # Get historical data for patterns
        historical_data = self.data_manager.load_user_weekly_data(user_id, num_weeks=4)

        context = f"USER PROFILE:\n"

        if profile:
            context += f"- Targets: {profile.targets}\n"
            context += f"- Preferences: {profile.preferences}\n"
            context += f"- Response Patterns: {profile.response_patterns}\n"

        if recent_recs:
            context += f"\nRECENT RECOMMENDATIONS:\n"
            for rec in recent_recs[:2]:  # Last 2 weeks
                context += f"Week {rec.week_start}:\n"
                for area, recommendation in rec.recommendations.items():
                    context += f"  {area}: {recommendation}\n"
                if rec.user_feedback.get('followed_exercise'):
                    context += f"  User followed exercise advice: {rec.user_feedback['followed_exercise']}\n"

        if historical_data:
            context += f"\nHISTORICAL PATTERNS:\n"
            # Calculate what user typically achieves
            avg_steps = sum([w.avg_daily_steps for w in historical_data]) / len(historical_data)
            avg_zone = sum([w.zone_minutes for w in historical_data]) / len(historical_data)
            context += f"- Typical daily steps: {avg_steps:.0f}\n"
            context += f"- Typical zone minutes: {avg_zone:.0f}\n"

        return context

    def _get_relevant_knowledge(self, analysis_data: Dict) -> str:
        """Get relevant knowledge from vector database based on priority areas"""

        priorities = analysis_data.get('priorities', [])

        if not priorities:
            # General health query
            search_query = "healthy lifestyle exercise nutrition sleep wellness"
        else:
            # Build query based on priority areas
            priority_areas = [p['area'] for p in priorities[:2]]  # Top 2 priorities
            search_query = " ".join(priority_areas) + " improvement recommendations health"

        # Search vector database
        relevant_docs = self.vectorstore.similarity_search(search_query, k=4)

        # Combine relevant knowledge
        knowledge = "\n---\n".join([doc.page_content for doc in relevant_docs])

        return knowledge[:2000]  # Limit context length

    def _build_comprehensive_prompt(self, user_id: str, analysis_data: Dict, user_context: str, knowledge_context: str) -> str:
        """Build comprehensive prompt with all context"""

        performance = analysis_data.get('performance', {})
        priorities = analysis_data.get('priorities', [])
        trends = analysis_data.get('trends', {})
        progress = analysis_data.get('progress', {})

        prompt = f"""<|system|>
You are an expert health and wellness AI assistant. You provide personalized recommendations based on comprehensive health data analysis.

IMPORTANT INSTRUCTIONS:
1. Provide recommendations in EXACTLY 4 sections: Food Recommendation, Physical Exercise, Sleep and Recovery, Overall Suggestion
2. Use bullet points with a), b), c), d) format
3. Base recommendations on the user's specific data, priorities, and historical patterns
4. Consider what has worked/not worked for this user before
5. Make recommendations progressive and achievable
6. End with "### End ###"

{user_context}

CURRENT WEEK ANALYSIS:
Overall Performance Score: {performance.get('overall', {}).get('score', 0):.1f}/100

PERFORMANCE BREAKDOWN:
- Steps: {performance.get('steps', {}).get('achievement_percent', 0):.1f}% of target
- Zone Minutes: {performance.get('zone_minutes', {}).get('achievement_percent', 0):.1f}% of target
- Sleep: {performance.get('sleep', {}).get('achievement_percent', 0):.1f}% of target
- Exercise Sessions: {performance.get('exercise_sessions', {}).get('achievement_percent', 0):.1f}% of target

PRIORITY AREAS NEEDING ATTENTION:
{self._format_priorities(priorities)}

TRENDS:
{self._format_trends(trends)}

PROGRESS FROM LAST WEEK:
{self._format_progress(progress)}

KNOWLEDGE BASE CONTEXT:
{knowledge_context}
<|/system|>

<|user|>
Based on my comprehensive health analysis above, provide personalized recommendations for the upcoming week. Focus on my priority areas while building on what has worked for me before.
<|/user|>

<|assistant|>
Response:
"""

        return prompt

    def _format_priorities(self, priorities: List[Dict]) -> str:
        """Format priority areas for prompt"""
        if not priorities:
            return "No major areas of concern - maintain current performance"

        formatted = ""
        for i, priority in enumerate(priorities[:3], 1):
            formatted += f"{i}. {priority['area'].replace('_', ' ').title()}: {', '.join(priority['reasons'])}\n"

        return formatted

    def _format_trends(self, trends: Dict) -> str:
        """Format trends for prompt"""
        if not trends:
            return "Insufficient data for trend analysis"

        formatted = ""
        for area, trend in trends.items():
            # Handle both string and dict trend values
            if isinstance(trend, dict):
                direction = trend.get('direction', 'stable')
                change = trend.get('percent_change', 0)
                formatted += f"- {area.replace('_', ' ').title()}: {direction}"
                if change != 0:
                    formatted += f" ({change:+.1f}%)"
            elif isinstance(trend, str):
                # Handle case where trend is just a string
                if trend in ['improving', 'declining', 'stable', 'insufficient_data']:
                    formatted += f"- {area.replace('_', ' ').title()}: {trend}"
                else:
                    formatted += f"- {area.replace('_', ' ').title()}: insufficient data"
            else:
                # Fallback for any other type
                formatted += f"- {area.replace('_', ' ').title()}: insufficient data"
            formatted += "\n"

        return formatted

    def _format_progress(self, progress: Dict) -> str:
        """Format progress for prompt"""
        if not progress:
            return "No previous week for comparison"

        formatted = ""
        for area, prog in progress.items():
            # Handle both string and dict progress values
            if isinstance(prog, dict):
                direction = prog.get('direction', 'maintained')
                change = prog.get('change', 0)
                formatted += f"- {area.replace('_', ' ').title()}: {direction}"
                if change != 0:
                    formatted += f" ({change:+.1f})"
            elif isinstance(prog, str):
                # Handle case where progress is just a string
                formatted += f"- {area.replace('_', ' ').title()}: {prog}"
            else:
                # Fallback
                formatted += f"- {area.replace('_', ' ').title()}: no data"
            formatted += "\n"

        return formatted

    def _generate_response(self, prompt: str) -> str:
        """Generate response using your model"""

        inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(self.model.device)

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=500,
                min_new_tokens=100,
                temperature=0.3,
                top_p=0.85,
                top_k=40,
                do_sample=True,
                repetition_penalty=1.1,
                pad_token_id=self.tokenizer.eos_token_id,
                eos_token_id=[
                    self.tokenizer.encode("### End ###")[0] if "### End ###" in self.tokenizer.get_vocab() else self.tokenizer.eos_token_id,
                    self.tokenizer.eos_token_id
                ],
                early_stopping=True,
                no_repeat_ngram_size=3
            )

        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract only the assistant's response
        if "<|assistant|>" in response:
            generated_text = response.split("<|assistant|>")[-1].strip()
        else:
            generated_text = response[len(prompt):].strip()

        # Clean up the response
        generated_text = re.sub(r'<\|.*?\|>', '', generated_text)
        generated_text = generated_text.strip()

        return generated_text

    def _parse_recommendations(self, raw_response: str) -> Dict:
        """Parse the LLM response into structured recommendations"""

        sections = {
            'food_recommendation': '',
            'physical_exercise': '',
            'sleep_and_recovery': '',
            'overall_suggestion': '',
            'raw_response': raw_response
        }

        # Extract each section
        patterns = {
            'food_recommendation': r'1\.\s*Food Recommendation[:\s]*(.*?)(?=2\.\s*Physical Exercise|$)',
            'physical_exercise': r'2\.\s*Physical Exercise[:\s]*(.*?)(?=3\.\s*Sleep and Recovery|$)',
            'sleep_and_recovery': r'3\.\s*Sleep and Recovery[:\s]*(.*?)(?=4\.\s*Overall Suggestion|$)',
            'overall_suggestion': r'4\.\s*Overall Suggestion[:\s]*(.*?)(?=### End ###|$)'
        }

        for section, pattern in patterns.items():
            match = re.search(pattern, raw_response, re.DOTALL | re.IGNORECASE)
            if match:
                sections[section] = match.group(1).strip()

        return sections

    def _update_user_learning(self, user_id: str, analysis_data: Dict, recommendations: Dict):
        """Update user learning patterns based on new recommendations"""

        # This will help improve future recommendations
        profile = self.data_manager.load_user_profile(user_id)

        if profile:
            # Track recommendation types given
            priorities = analysis_data.get('priorities', [])
            if priorities:
                top_priority = priorities[0]['area']
                if top_priority not in profile.response_patterns.get('areas_worked_on', []):
                    if 'areas_worked_on' not in profile.response_patterns:
                        profile.response_patterns['areas_worked_on'] = []
                    profile.response_patterns['areas_worked_on'].append(top_priority)

            # Save updated profile
            self.data_manager.save_user_profile(profile)

    def update_recommendation_feedback(self, user_id: str, week_start: str, feedback: Dict):
        """Update feedback on recommendations to improve future suggestions"""

        # Load the recommendation
        recommendations = self.data_manager.load_user_recommendations(user_id, num_weeks=4)

        for rec in recommendations:
            if rec.week_start == week_start:
                rec.update_feedback(feedback)
                self.data_manager.save_recommendations(rec)

                # Update user profile learning patterns
                profile = self.data_manager.load_user_profile(user_id)
                if profile:
                    # Update response patterns based on feedback
                    if feedback.get('followed_exercise'):
                        profile.response_patterns['follows_exercise_recs'] = min(1.0,
                            profile.response_patterns.get('follows_exercise_recs', 0.5) + 0.1)

                    if feedback.get('followed_nutrition'):
                        profile.response_patterns['follows_nutrition_recs'] = min(1.0,
                            profile.response_patterns.get('follows_nutrition_recs', 0.5) + 0.1)

                    if feedback.get('followed_sleep'):
                        profile.response_patterns['follows_sleep_recs'] = min(1.0,
                            profile.response_patterns.get('follows_sleep_recs', 0.5) + 0.1)

                    self.data_manager.save_user_profile(profile)
                break

# ========================================================================
# STEP 3 USAGE EXAMPLE (COLAB FIXED)
# ========================================================================

def example_llm_processing():
    """Example of using the enhanced LLM processor - CORRECTED FOR COLAB"""

    # No imports needed - classes are already defined in previous cells

    # Initialize components
    data_manager = HealthDataManager()
    analytics = HealthAnalytics(data_manager)

    # Create user profile first
    user_id = "user_123"
    profile = UserProfile(user_id)
    profile.update_targets({
        'daily_steps': 10000,
        'weekly_zone_minutes': 150,
        'daily_sleep_hours': 8,
        'weekly_exercise_sessions': 3,
        'daily_calories': 2000
    })
    data_manager.save_user_profile(profile)
    print(f"✅ Created user profile for {user_id}")

    # Create some historical data for better analysis
    historical_week = WeeklyHealthData(user_id, "2024-01-22", {
        'total_steps': 62000,
        'zone_minutes': 100,
        'sleep_hours': [6.8, 6.0, 7.5, 6.5, 6.2, 8.0, 7.2],
        'exercise_sessions': [{'type': 'walking', 'duration': 30}],
        'mood_scores': [6, 5, 7, 6, 4, 7, 6],
        'food_data': []
    })
    data_manager.save_weekly_data(historical_week)
    print("✅ Created historical data")

    try:
        # Initialize LLM processor with your model paths
        llm_processor = HealthLLMProcessor(
            base_model_id="ContactDoctor/Bio-Medical-Llama-3-8B",
            adapter_path="AnjaliNV/WellBeing_LLM",
            vector_db_path="/content/drive/MyDrive/rag_index",  # Your vector DB path
            data_manager=data_manager
        )
        print("✅ LLM processor initialized")

        # Current week data
        current_week_data = WeeklyHealthData(user_id, "2024-01-29", {
            'total_steps': 58000,  # Below target
            'zone_minutes': 90,    # Below target
            'sleep_hours': [6.5, 6.2, 7.8, 6.8, 6.0, 8.2, 7.5],
            'exercise_sessions': [
                {'type': 'yoga', 'duration': 45}
            ],
            'mood_scores': [6, 5, 7, 6, 5, 8, 7],
            'food_data': []
        })
        data_manager.save_weekly_data(current_week_data)
        print("✅ Created current week data")

        # Analyze the week
        analysis = analytics.analyze_weekly_performance(user_id, current_week_data)
        print("✅ Analysis completed")

        # Check for analysis errors
        if 'error' in analysis:
            print(f"❌ Analysis error: {analysis['error']}")
            return None

        # Generate recommendations
        print("🤖 Generating recommendations with LLM...")
        recommendations = llm_processor.generate_weekly_recommendations(user_id, analysis)
        print("✅ Recommendations generated")

        print("\n" + "="*60)
        print("=== COMPREHENSIVE WEEKLY RECOMMENDATIONS ===")
        print("="*60)
        print(f"👤 User: {user_id}")
        print(f"📅 Week: {current_week_data.week_start}")
        print(f"🎯 Overall Score: {analysis['performance']['overall']['score']:.1f}/100")

        print("\n📊 PRIORITY AREAS:")
        if analysis['priorities']:
            for priority in analysis['priorities'][:2]:
                print(f"• {priority['area'].replace('_', ' ').title()}: {', '.join(priority['reasons'])}")
        else:
            print("• No major priority areas - maintain current performance")

        print("\n🍎 FOOD RECOMMENDATIONS:")
        print(recommendations.get('food_recommendation', 'No specific food recommendations available'))

        print("\n💪 EXERCISE RECOMMENDATIONS:")
        print(recommendations.get('physical_exercise', 'No specific exercise recommendations available'))

        print("\n😴 SLEEP & RECOVERY:")
        print(recommendations.get('sleep_and_recovery', 'No specific sleep recommendations available'))

        print("\n🎯 OVERALL SUGGESTIONS:")
        print(recommendations.get('overall_suggestion', 'No specific overall suggestions available'))

        # Save the recommendations
        weekly_recs = WeeklyRecommendations(user_id, current_week_data.week_start, {
            'food': recommendations.get('food_recommendation', ''),
            'exercise': recommendations.get('physical_exercise', ''),
            'sleep': recommendations.get('sleep_and_recovery', ''),
            'overall': recommendations.get('overall_suggestion', '')
        })

        data_manager.save_recommendations(weekly_recs)
        print("\n✅ Recommendations saved for future learning")

        return recommendations

    except Exception as e:
        print(f"❌ Error during LLM processing: {str(e)}")
        import traceback
        traceback.print_exc()

        # Return a fallback recommendation structure
        return {
            'food_recommendation': 'Error generating recommendations - please try again',
            'physical_exercise': 'Error generating recommendations - please try again',
            'sleep_and_recovery': 'Error generating recommendations - please try again',
            'overall_suggestion': 'Error generating recommendations - please try again',
            'error': str(e)
        }

# Test the LLM processing (optional - uncomment to test)
# if __name__ == "__main__":
#     recommendations = example_llm_processing()

In [ ]:
# ========================================================================
# STEP 4: Complete Integrated Health Analytics System
# ========================================================================
import gradio as gr
from datetime import datetime, timedelta
import json
from typing import Dict, List

class HealthAnalyticsSystem:
    """Complete health analytics system with memory and learning"""

    def __init__(self, base_model_id: str, adapter_path: str, vector_db_path: str, data_dir: str = "health_data"):
        """Initialize the complete system"""

        print("🚀 Initializing Health Analytics System...")

        # No imports needed in Colab - classes already defined in previous cells

        # Initialize components
        self.data_manager = HealthDataManager(data_dir)
        self.analytics = HealthAnalytics(self.data_manager)
        self.llm_processor = HealthLLMProcessor(base_model_id, adapter_path, vector_db_path, self.data_manager)

        # Store classes for creating objects (reference to global classes)
        self.UserProfile = UserProfile
        self.WeeklyHealthData = WeeklyHealthData
        self.WeeklyRecommendations = WeeklyRecommendations

        print("✅ Health Analytics System ready!")

    def process_weekly_data(self, user_id: str, week_data: Dict, targets: Dict = None) -> Dict:
        """Main function to process weekly health data and generate recommendations"""

        # Ensure user profile exists
        profile = self.data_manager.load_user_profile(user_id)
        if not profile:
            profile = self.UserProfile(user_id)
            if targets:
                profile.update_targets(targets)
            self.data_manager.save_user_profile(profile)
            print(f"Created new user profile for {user_id}")
        elif targets:
            profile.update_targets(targets)
            self.data_manager.save_user_profile(profile)

        # Create weekly data object
        week_start = week_data.get('week_start', datetime.now().strftime('%Y-%m-%d'))
        weekly_health_data = self.WeeklyHealthData(user_id, week_start, week_data)

        # Save weekly data
        self.data_manager.save_weekly_data(weekly_health_data)

        # Perform comprehensive analysis
        analysis = self.analytics.analyze_weekly_performance(user_id, weekly_health_data)

        # Generate recommendations using LLM
        recommendations = self.llm_processor.generate_weekly_recommendations(user_id, analysis)

        # Save recommendations
        weekly_recs = self.WeeklyRecommendations(user_id, week_start, {
            'food': recommendations.get('food_recommendation', ''),
            'exercise': recommendations.get('physical_exercise', ''),
            'sleep': recommendations.get('sleep_and_recovery', ''),
            'overall': recommendations.get('overall_suggestion', ''),
            'analysis_summary': self._create_analysis_summary(analysis)
        })

        self.data_manager.save_recommendations(weekly_recs)

        # Return comprehensive result
        return {
            'user_id': user_id,
            'week_start': week_start,
            'analysis': analysis,
            'recommendations': recommendations,
            'insights': self.analytics.generate_insights(analysis),
            'week_summary': self._create_week_summary(weekly_health_data, analysis)
        }

    def provide_feedback(self, user_id: str, week_start: str, feedback: Dict):
        """Allow users to provide feedback on recommendations"""

        self.llm_processor.update_recommendation_feedback(user_id, week_start, feedback)
        print(f"Feedback updated for {user_id}, week {week_start}")

    def get_user_progress(self, user_id: str, weeks: int = 8) -> Dict:
        """Get comprehensive user progress over time"""

        progress_summary = self.data_manager.get_user_progress_summary(user_id)
        recent_recommendations = self.data_manager.load_user_recommendations(user_id, weeks)

        return {
            'progress_summary': progress_summary,
            'recent_recommendations': [rec.to_dict() for rec in recent_recommendations],
            'trend_analysis': self._analyze_long_term_trends(user_id, weeks)
        }

    def _create_analysis_summary(self, analysis: Dict) -> str:
        """Create a summary of the analysis"""

        performance = analysis.get('performance', {})
        overall_score = performance.get('overall', {}).get('score', 0)

        summary = f"Overall Health Score: {overall_score:.1f}/100\n"

        # Performance breakdown
        summary += "\nPerformance Breakdown:\n"
        for area, perf in performance.items():
            if area != 'overall':
                summary += f"• {area.replace('_', ' ').title()}: {perf.get('achievement_percent', 0):.1f}%\n"

        # Priority areas
        priorities = analysis.get('priorities', [])
        if priorities:
            summary += f"\nTop Priority: {priorities[0]['area'].replace('_', ' ').title()}\n"

        return summary

    def _create_week_summary(self, weekly_data: WeeklyHealthData, analysis: Dict) -> str:
        """Create a human-readable week summary"""

        summary = f"Week of {weekly_data.week_start}:\n"
        summary += f"• Steps: {weekly_data.total_steps:,} ({weekly_data.avg_daily_steps:.0f}/day)\n"
        summary += f"• Zone Minutes: {weekly_data.zone_minutes}\n"
        summary += f"• Average Sleep: {weekly_data.avg_sleep:.1f} hours\n"
        summary += f"• Exercise Sessions: {len(weekly_data.exercise_sessions)}\n"
        summary += f"• Average Mood: {weekly_data.avg_mood:.1f}/10\n"

        return summary

    def _analyze_long_term_trends(self, user_id: str, weeks: int) -> Dict:
        """Analyze long-term trends"""

        weekly_data = self.data_manager.load_user_weekly_data(user_id, weeks)

        if len(weekly_data) < 3:
            return {"message": "Insufficient data for trend analysis"}

        # Calculate trends
        weeks_data = list(reversed(weekly_data))  # Chronological order

        trends = {
            'steps_trend': self._calculate_simple_trend([w.avg_daily_steps for w in weeks_data]),
            'zone_trend': self._calculate_simple_trend([w.zone_minutes for w in weeks_data]),
            'sleep_trend': self._calculate_simple_trend([w.avg_sleep for w in weeks_data]),
            'mood_trend': self._calculate_simple_trend([w.avg_mood for w in weeks_data if w.avg_mood > 0])
        }

        return trends

    def _calculate_simple_trend(self, data: List[float]) -> str:
        """Calculate simple trend direction"""
        if len(data) < 3:
            return "insufficient_data"

        recent_avg = sum(data[-3:]) / 3
        earlier_avg = sum(data[:3]) / 3

        change_percent = ((recent_avg - earlier_avg) / earlier_avg) * 100 if earlier_avg > 0 else 0

        if change_percent > 5:
            return "improving"
        elif change_percent < -5:
            return "declining"
        else:
            return "stable"

# ========================================================================
# STEP 4: Enhanced Gradio Interface
# ========================================================================

class HealthAnalyticsInterface:
    """Enhanced Gradio interface for the health analytics system"""

    def __init__(self, health_system: HealthAnalyticsSystem):
        self.health_system = health_system
        self.current_user_id = "default_user"

    def create_interface(self):
        """Create the Gradio interface"""

        with gr.Blocks(title="Health Analytics AI", theme=gr.themes.Soft()) as demo:
            gr.Markdown("# 🏥 Health Analytics AI Assistant")
            gr.Markdown("Analyze your weekly health data and get personalized recommendations")

            with gr.Tab("📊 Weekly Analysis"):
                with gr.Row():
                    with gr.Column():
                        user_id_input = gr.Textbox(
                            label="User ID",
                            value="user_123",
                            placeholder="Enter your user ID"
                        )

                        week_start_input = gr.Textbox(
                            label="Week Start Date",
                            value=datetime.now().strftime('%Y-%m-%d'),
                            placeholder="YYYY-MM-DD"
                        )

                        with gr.Row():
                            steps_input = gr.Number(
                                label="Total Steps (Week)",
                                value=65000,
                                minimum=0
                            )
                            zone_minutes_input = gr.Number(
                                label="Zone Minutes (Week)",
                                value=120,
                                minimum=0
                            )

                        sleep_input = gr.Textbox(
                            label="Daily Sleep Hours",
                            value="7.5,6.8,8.2,7.0,6.5,8.5,7.8",
                            placeholder="Enter 7 values separated by commas"
                        )

                        mood_input = gr.Textbox(
                            label="Daily Mood Scores (1-10)",
                            value="7,6,8,7,5,8,7",
                            placeholder="Enter 7 values separated by commas"
                        )

                        exercise_input = gr.Number(
                            label="Exercise Sessions",
                            value=2,
                            minimum=0
                        )

                        calories_input = gr.Number(
                            label="Total Calories (Week)",
                            value=14000,
                            minimum=0
                        )

                        analyze_btn = gr.Button("🔍 Analyze Health Data", variant="primary")

                    with gr.Column():
                        analysis_output = gr.Markdown("Click 'Analyze Health Data' to see your personalized recommendations")

            with gr.Tab("📈 Progress Tracking"):
                with gr.Row():
                    progress_user_input = gr.Textbox(
                        label="User ID",
                        value="user_123"
                    )
                    weeks_input = gr.Number(
                        label="Number of Weeks",
                        value=4,
                        minimum=1,
                        maximum=12
                    )
                    progress_btn = gr.Button("📊 View Progress")

                progress_output = gr.Markdown("Enter user ID and click 'View Progress'")

            with gr.Tab("💬 Feedback"):
                with gr.Row():
                    feedback_user_input = gr.Textbox(
                        label="User ID",
                        value="user_123"
                    )
                    feedback_week_input = gr.Textbox(
                        label="Week Date",
                        value=datetime.now().strftime('%Y-%m-%d')
                    )

                with gr.Row():
                    followed_exercise = gr.Checkbox(label="Followed Exercise Recommendations")
                    followed_nutrition = gr.Checkbox(label="Followed Nutrition Recommendations")
                    followed_sleep = gr.Checkbox(label="Followed Sleep Recommendations")

                with gr.Row():
                    difficulty_rating = gr.Slider(
                        label="Difficulty Level (1-5)",
                        minimum=1,
                        maximum=5,
                        value=3
                    )
                    effectiveness_rating = gr.Slider(
                        label="Effectiveness (1-5)",
                        minimum=1,
                        maximum=5,
                        value=3
                    )

                feedback_notes = gr.Textbox(
                    label="Additional Notes",
                    placeholder="Any additional feedback about the recommendations..."
                )

                feedback_btn = gr.Button("💌 Submit Feedback")
                feedback_output = gr.Markdown("Submit your feedback to help improve future recommendations")

            # Event handlers
            analyze_btn.click(
                fn=self.analyze_weekly_data,
                inputs=[
                    user_id_input, week_start_input, steps_input, zone_minutes_input,
                    sleep_input, mood_input, exercise_input, calories_input
                ],
                outputs=[analysis_output]
            )

            progress_btn.click(
                fn=self.view_progress,
                inputs=[progress_user_input, weeks_input],
                outputs=[progress_output]
            )

            feedback_btn.click(
                fn=self.submit_feedback,
                inputs=[
                    feedback_user_input, feedback_week_input, followed_exercise,
                    followed_nutrition, followed_sleep, difficulty_rating,
                    effectiveness_rating, feedback_notes
                ],
                outputs=[feedback_output]
            )

        return demo

    def analyze_weekly_data(self, user_id, week_start, steps, zone_minutes, sleep_str, mood_str, exercise_sessions, calories):
        """Process weekly data and return analysis"""

        try:
            # Parse sleep and mood data
            sleep_hours = [float(x.strip()) for x in sleep_str.split(',') if x.strip()]
            mood_scores = [float(x.strip()) for x in mood_str.split(',') if x.strip()]

            # Create week data
            week_data = {
                'week_start': week_start,
                'total_steps': int(steps),
                'zone_minutes': int(zone_minutes),
                'sleep_hours': sleep_hours,
                'mood_scores': mood_scores,
                'exercise_sessions': [{'type': 'general', 'duration': 30} for _ in range(int(exercise_sessions))],
                'food_data': [{'calories': calories / 7} for _ in range(7)]  # Distribute calories
            }

            # Default targets
            targets = {
                'daily_steps': 10000,
                'weekly_zone_minutes': 150,
                'daily_sleep_hours': 8,
                'weekly_exercise_sessions': 3,
                'daily_calories': 2000
            }

            # Process the data
            result = self.health_system.process_weekly_data(user_id, week_data, targets)

            # Format the output
            return self._format_analysis_output(result)

        except Exception as e:
            return f"❌ Error processing data: {str(e)}\n\nPlease check your input format."

    def view_progress(self, user_id, weeks):
        """View user progress over time"""

        try:
            progress = self.health_system.get_user_progress(user_id, int(weeks))
            return self._format_progress_output(progress)
        except Exception as e:
            return f"❌ Error retrieving progress: {str(e)}"

    def submit_feedback(self, user_id, week_date, followed_exercise, followed_nutrition, followed_sleep, difficulty, effectiveness, notes):
        """Submit feedback on recommendations"""

        try:
            feedback = {
                'followed_exercise': followed_exercise,
                'followed_nutrition': followed_nutrition,
                'followed_sleep': followed_sleep,
                'difficulty_level': difficulty,
                'effectiveness': effectiveness,
                'notes': notes
            }

            self.health_system.provide_feedback(user_id, week_date, feedback)

            return f"""✅ **Feedback Submitted Successfully!**

**Week:** {week_date}
**Followed Recommendations:**
- Exercise: {'✅' if followed_exercise else '❌'}
- Nutrition: {'✅' if followed_nutrition else '❌'}
- Sleep: {'✅' if followed_sleep else '❌'}

**Ratings:**
- Difficulty: {difficulty}/5
- Effectiveness: {effectiveness}/5

**Notes:** {notes if notes else 'None'}

Your feedback helps improve future recommendations!"""

        except Exception as e:
            return f"❌ Error submitting feedback: {str(e)}"

    def _format_analysis_output(self, result: Dict) -> str:
        """Format the analysis output for display"""

        analysis = result['analysis']
        recommendations = result['recommendations']
        insights = result['insights']

        output = f"""# 🏥 Health Analysis for {result['user_id']}

## 📊 **Week Summary** ({result['week_start']})
{result['week_summary']}

## 🎯 **Overall Health Score: {analysis['performance']['overall']['score']:.1f}/100**

## 📈 **Key Insights**
"""

        for insight in insights:
            output += f"- {insight}\n"

        output += "\n## 🏆 **Performance Breakdown**\n"
        for area, perf in analysis['performance'].items():
            if area != 'overall':
                status_emoji = {'excellent': '🟢', 'good': '🟡', 'fair': '🟠', 'needs_improvement': '🔴'}.get(perf['status'], '⚪')
                output += f"- **{area.replace('_', ' ').title()}:** {perf['achievement_percent']:.1f}% {status_emoji}\n"

        if analysis['priorities']:
            output += "\n## ⚠️ **Priority Areas**\n"
            for priority in analysis['priorities'][:3]:
                output += f"- **{priority['area'].replace('_', ' ').title()}:** {', '.join(priority['reasons'])}\n"

        output += f"\n## 🍎 **Food Recommendations**\n{recommendations.get('food_recommendation', 'No specific recommendations')}\n"
        output += f"\n## 💪 **Exercise Recommendations**\n{recommendations.get('physical_exercise', 'No specific recommendations')}\n"
        output += f"\n## 😴 **Sleep & Recovery**\n{recommendations.get('sleep_and_recovery', 'No specific recommendations')}\n"
        output += f"\n## 🎯 **Overall Suggestions**\n{recommendations.get('overall_suggestion', 'No specific recommendations')}\n"

        output += "\n---\n*💡 Tip: Use the Feedback tab to let us know how these recommendations work for you!*"

        return output

    def _format_progress_output(self, progress: Dict) -> str:
        """Format progress output for display"""

        summary = progress.get('progress_summary', {})
        trends = progress.get('trend_analysis', {})

        if summary.get('message'):
            return f"ℹ️ {summary['message']}"

        output = f"""# 📈 Progress Report

## 📊 **Tracking Summary**
- **Weeks Tracked:** {summary.get('weeks_tracked', 0)}
- **Data Points Available:** {len(summary.get('steps_trend', []))}

## 📈 **Long-term Trends**
"""

        trend_emojis = {'improving': '📈', 'declining': '📉', 'stable': '➡️', 'insufficient_data': '❓'}

        for area, trend in trends.items():
            emoji = trend_emojis.get(trend, '❓')
            output += f"- **{area.replace('_', ' ').title()}:** {trend.replace('_', ' ').title()} {emoji}\n"

        if summary.get('latest_week'):
            latest = summary['latest_week']
            output += f"""
## 📅 **Latest Week Data**
- **Week:** {latest.get('week_start')}
- **Daily Steps:** {latest.get('avg_daily_steps', 0):.0f}
- **Zone Minutes:** {latest.get('zone_minutes', 0)}
- **Sleep:** {latest.get('avg_sleep', 0):.1f} hours
- **Mood:** {latest.get('avg_mood', 0):.1f}/10
"""

        # Recent recommendations summary
        recent_recs = progress.get('recent_recommendations', [])
        if recent_recs:
            output += f"\n## 📝 **Recent Recommendations** (Last {len(recent_recs)} weeks)\n"
            for i, rec in enumerate(recent_recs[:3], 1):
                output += f"{i}. **Week {rec['week_start']}:** {rec['recommendations'].get('overall', 'No summary available')[:100]}...\n"

        return output

# ========================================================================
# STEP 4: Main Application
# ========================================================================

def create_health_analytics_app():
    """Create and launch the complete health analytics application"""

    # Initialize the health analytics system
    health_system = HealthAnalyticsSystem(
        base_model_id="ContactDoctor/Bio-Medical-Llama-3-8B",
        adapter_path="AnjaliNV/WellBeing_LLM",
        vector_db_path="/content/drive/MyDrive/rag_index",  # Your vector DB path
        data_dir="health_data"
    )

    # Create the interface
    interface = HealthAnalyticsInterface(health_system)
    demo = interface.create_interface()

    return demo, health_system

# ========================================================================
# STEP 4: Usage Example with Sample Data
# ========================================================================

def run_example_analysis():
    """Run example analysis to demonstrate the system - CORRECTED FOR COLAB"""

    print("🧪 Running example analysis...")

    # Create the system
    demo, health_system = create_health_analytics_app()

    # Example: Simulate 3 weeks of data for a user
    user_id = "demo_user"

    # Week 1 - Baseline
    week1_data = {
        'week_start': '2024-01-15',
        'total_steps': 58000,
        'zone_minutes': 90,
        'sleep_hours': [6.5, 6.2, 7.8, 6.8, 6.0, 8.2, 7.5],
        'exercise_sessions': [{'type': 'yoga', 'duration': 45}],
        'mood_scores': [6, 5, 7, 6, 5, 8, 7],
        'food_data': [{'calories': 400} for _ in range(21)]  # 3 meals/day
    }

    targets = {
        'daily_steps': 10000,
        'weekly_zone_minutes': 150,
        'daily_sleep_hours': 8,
        'weekly_exercise_sessions': 3,
        'daily_calories': 2000
    }

    try:
        result1 = health_system.process_weekly_data(user_id, week1_data, targets)
        print(f"✅ Week 1 processed - Score: {result1['analysis']['performance']['overall']['score']:.1f}/100")

        # Week 2 - Some improvement
        week2_data = {
            'week_start': '2024-01-22',
            'total_steps': 68000,  # Improved
            'zone_minutes': 130,   # Improved
            'sleep_hours': [7.2, 6.8, 8.1, 7.3, 6.8, 8.0, 7.6],
            'exercise_sessions': [
                {'type': 'running', 'duration': 30},
                {'type': 'yoga', 'duration': 45}
            ],
            'mood_scores': [7, 6, 8, 7, 6, 8, 7],
            'food_data': [{'calories': 400} for _ in range(21)]
        }

        result2 = health_system.process_weekly_data(user_id, week2_data)
        print(f"✅ Week 2 processed - Score: {result2['analysis']['performance']['overall']['score']:.1f}/100")

        # Week 3 - Further improvement
        week3_data = {
            'week_start': '2024-01-29',
            'total_steps': 74000,  # Further improved
            'zone_minutes': 155,   # Target reached!
            'sleep_hours': [7.8, 7.5, 8.2, 7.7, 7.2, 8.1, 7.9],
            'exercise_sessions': [
                {'type': 'running', 'duration': 35},
                {'type': 'cycling', 'duration': 40},
                {'type': 'yoga', 'duration': 45}
            ],
            'mood_scores': [8, 7, 8, 8, 7, 9, 8],
            'food_data': [{'calories': 400} for _ in range(21)]
        }

        result3 = health_system.process_weekly_data(user_id, week3_data)
        print(f"✅ Week 3 processed - Score: {result3['analysis']['performance']['overall']['score']:.1f}/100")

        # Show progress
        progress = health_system.get_user_progress(user_id, 3)
        print(f"\n📈 Progress Summary:")
        print(f"- Weeks tracked: {progress['progress_summary']['weeks_tracked']}")
        print(f"- Steps trend: {progress['trend_analysis'].get('steps_trend', 'N/A')}")
        print(f"- Zone minutes trend: {progress['trend_analysis'].get('zone_trend', 'N/A')}")
        print(f"- Sleep trend: {progress['trend_analysis'].get('sleep_trend', 'N/A')}")

        # Simulate feedback
        health_system.provide_feedback(user_id, '2024-01-22', {
            'followed_exercise': True,
            'followed_nutrition': True,
            'followed_sleep': False,
            'difficulty_level': 3,
            'effectiveness': 4,
            'notes': 'Exercise recommendations were great! Sleep tips were hard to follow.'
        })
        print("✅ Feedback submitted")

        print("\n🚀 System ready! Use demo.launch() to start the interface.")
        return demo, health_system

    except Exception as e:
        print(f"❌ Error during example analysis: {str(e)}")
        import traceback
        traceback.print_exc()
        return demo, health_system

if __name__ == "__main__":
    # Run the example
    demo, health_system = run_example_analysis()

    # Launch the interface
    # demo.launch(debug=True, share=True)

In [ ]:
demo.launch()